In [18]:
import pandas as pd

In [19]:
import numpy as np
from sklearn.neighbors import DistanceMetric
from scipy.cluster.hierarchy import dendrogram, linkage

def gower_distance(X):
    """
    This function expects a pandas dataframe as input
    The data frame is to contain the features along the columns. Based on these features a
    distance matrix will be returned which will contain the pairwise gower distance between the rows
    All variables of object type will be treated as nominal variables and the others will be treated as 
    numeric variables.
    Distance metrics used for:
    Nominal variables: Dice distance (https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient)
    Numeric variables: Manhattan distance normalized by the range of the variable (https://en.wikipedia.org/wiki/Taxicab_geometry)
    """
    individual_variable_distances = []

    for i in range(X.shape[1]):
        feature = X.iloc[:,[i]]
        if feature.dtypes[0] == np.object:
            feature_dist = DistanceMetric.get_metric('dice').pairwise(pd.get_dummies(feature))
        else:
            feature_dist = DistanceMetric.get_metric('manhattan').pairwise(feature) / np.ptp(feature.values)

        individual_variable_distances.append(feature_dist)

    return np.array(individual_variable_distances).mean(0)

In [20]:
df = pd.read_csv('murano_dmp_pull2_2.csv')

new_df = df[['taxonomy_entry_category','taxonomy_entry_name','seg1','seg1_uu','index']]

# print(new_df.columns)

row_dict_arr = []

for i,row in new_df.iterrows():
    row_dict = {}
    for col_name in new_df.columns:
        row_dict[col_name] = row[col_name]
#     row_dict_arr += int(row['seg1_uu'])*[row_dict]
    row_dict_arr += int(1)*[row_dict]
result_df = pd.DataFrame(row_dict_arr)

result_df = result_df.sort_values('seg1_uu', ascending = False)

segs = result_df.seg1.unique()

print(segs)

X = result_df[['seg1_uu', 'index', 'taxonomy_entry_name']]

seg1_uu_median = result_df['seg1_uu'].median()

index_median = 110



print(index_median)


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['pragmatic_buyers' 'parent_helpers' 'big_life_events' 'status_conscious'
 'random']
110


In [21]:
cut_df = X.loc[(X['seg1_uu'] >= seg1_uu_median)]

cut_df = cut_df.loc[(cut_df['index'] >= index_median)]

print(cut_df.count)

<bound method DataFrame.count of         seg1_uu  index                                taxonomy_entry_name
280518    49370    110  Known Consumers who enjoy Electronics - Computers
280517    49317    113                                          Homeowner
280516    49308    113                                          Homeowner
280515    49247    110  Known Consumers who enjoy Electronics - Computers
280514    49119    113                        Known Mail Order Responders
280513    49115    113                            Known Mail Order Buyers
280510    48912    113                          Known owners of Computers
280509    48887    112                        Known Mail Order Responders
280508    48886    112                            Known Mail Order Buyers
280507    48873    112                  Known Consumers who enjoy Reading
280506    48807    110  Demographics -> D25 - Dwelling Type Single Fam...
280505    48775    113                          Known owners of Computers
28050

In [29]:
# Z = gower_distance(X)

import numpy 
import pandas
from  matplotlib import pyplot
import seaborn
import json
seaborn.set(style='ticks')

taxonomy_names = list(X.taxonomy_entry_name.unique())


df = cut_df


print(json.dumps(df.to_json(), indent=4), file=open("axciom_nissan_cut_data.json", "a"))
# fg = seaborn.FacetGrid(data=df, hue='taxonomy_entry_name', hue_order=taxonomy_names, aspect=1.61)
# fg.map(pyplot.scatter, 'seg1_uu', 'index').add_legend()
# from scipy.cluster.hierarchy import dendrogram, linkage
# from scipy.spatial.distance import squareform

# import matplotlib.pyplot as plt


# def fancy_dendrogram(*args, **kwargs):
#     max_d = kwargs.pop('max_d', None)
#     if max_d and 'color_threshold' not in kwargs:
#         kwargs['color_threshold'] = max_d
#     annotate_above = kwargs.pop('annotate_above', 0)

#     ddata = dendrogram(*args, **kwargs)

#     if not kwargs.get('no_plot', False):
#         plt.title('Hierarchical Clustering Dendrogram (truncated)')
#         plt.xlabel('sample index or (cluster size)')
#         plt.ylabel('distance')
#         for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
#             x = 0.5 * sum(i[1:3])
#             y = d[1]
#             if y > annotate_above:
#                 plt.plot(x, y, 'o', c=c)
#                 plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
#                              textcoords='offset points',
#                              va='top', ha='center')
#         if max_d:
#             plt.axhline(y=max_d, c='k')
#     return ddata

# dists = squareform(Z)
# linkage_matrix = linkage(dists, "ward")

# from scipy.cluster.hierarchy import fcluster
# # fcluster(linkage_matrix, 4, criterion='maxclust')

# print(fcluster(linkage_matrix, 4, criterion='maxclust'))